In [ ]:
from __future__ import division, print_function

import collections
import csv
import datetime
import xml.etree.ElementTree as ET

import numpy as np
import pandas as pd

from datetime import datetime, timedelta
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
import pickle
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Implementation

In [ ]:
from LSTM_functions import *

In [ ]:
# create the test dataset
filepath = f"../OhioT1DM/2018/train/559-ws-training.xml"

# repeat the same process for the test dataset
glucose = read_ohio(filepath, "glucose_level", True)
glucose_df = transfer_into_table(glucose)
segments = segement_data_as_15min(glucose_df)
meal = add_meal_segments(filepath)
bolus = add_bolus_segments(filepath, meal)

steps = read_ohio(filepath, "basis_steps", True)
flattened_steps_data = [item[0] for item in steps]
step_df = pd.DataFrame(flattened_steps_data)
step_updated_segments = optimize_step_processing(bolus, step_df)

In [ ]:
import glob
import os

def process_all_training_files(directory_path):
    # Get all XML files in the directory
    xml_files = glob.glob(os.path.join(directory_path, "*-ws-training.xml"))
    
    all_processed_data = []
    
    for filepath in xml_files:
        try:
            # Process each file
            glucose = read_ohio(filepath, "glucose_level", True)
            glucose_df = transfer_into_table(glucose)
            segments = segement_data_as_15min(glucose_df)
            # meal = add_meal_segments(filepath)
            bolus = add_bolus_segments(filepath, segments)

            # steps = read_ohio(filepath, "basis_steps", True)
            # flattened_steps_data = [item[0] for item in steps]
            # step_df = pd.DataFrame(flattened_steps_data)
            # step_updated_segments = optimize_step_processing(bolus, step_df)
            
            # Add to list of processed data
            all_processed_data.append({
                'filepath': filepath,
                'segments': bolus
            })
            
            print(f"Successfully processed {filepath}")
            
        except Exception as e:
            print(f"Error processing {filepath}: {str(e)}")
    
    return all_processed_data

# Usage
directory_path = "../OhioT1DM/2018/train/"
training_data = process_all_training_files(directory_path)

In [ ]:
# create a dictionary such that the key is the segment number + i and the value is the segment data
segment_dict = {}
count = 0

segment_name_list = []
segment_data_list= []
for i in training_data: 
    count += 1
    for j in i['segments']:
        segment_dict[str(count)+j] = i['segments'][j]


In [ ]:
segment_dict

In [ ]:
# # Save the processed data CAREFULL!!!
# # Specify the file name
# filename = './processed_data/BIG_training_data_onlyCGM.pkl'
# # Save the dictionary to a file
# if not os.path.exists(filename):
#     open(filename, 'wb').close()
# # Save the dictionary to a file
# with open(filename, 'wb') as f:
#     pickle.dump(segment_dict, f)


# Test

In [ ]:
# create the test dataset
dirpath = f"../OhioT1DM/2018/test/"

for filename in os.listdir(dirpath):
    filepath = os.path.join(dirpath,filename)

    # repeat the same process for the test dataset
    glucose = read_ohio(filepath, "glucose_level", True)
    glucose_df = transfer_into_table(glucose)


    segments = segement_data_as_15min(glucose_df)
    # meal = add_meal_segments(filepath)
    bolus = add_bolus_segments(filepath, segments)

    # steps = read_ohio(filepath, "basis_steps", True)
    # flattened_steps_data = [item[0] for item in steps]
    # step_df = pd.DataFrame(flattened_steps_data)
    # # step_updated_segments = optimize_step_processing(bolus, step_df)
    # filename = './processed_data/{}_test_combined_segments_noshrink_ONLYCGM.pkl'.format(filename.split('-')[0])

    # # Save the dictionary to a file
    # if not os.path.exists(filename):
    #     open(filename, 'wb').close()

    # # Save the dictionary to the file
    # with open(filename, 'wb') as f:
    #     pickle.dump(bolus, f)

In [ ]:
# Save the processed data CAREFULL!!!
# Specify the file name
import os

filename = './processed_data/559_test_combined_segments_noshrink.pkl'

# Save the dictionary to a file
if not os.path.exists(filename):
    open(filename, 'wb').close()

# Save the dictionary to the file
with open(filename, 'wb') as f:
    pickle.dump(step_updated_segments, f)